# Reporting Applied Prediction Probabilities - Replication Data- (for STREAMLINE)
The pipeline outputs pickled objects with all the metric results during the initial testing evaluation of trained models as well as following application of trained models to additional hold out replication data (i.e. apply data).

This notebook grabs these prediction probabilities on the replication data and reports them as .csv files for each algorithm and CV partition pair (i.e for each of the CV trained models).  Unlike the initial testing evaluation, here prediction probabilities for all instances in the replication data are reported rather than just for those in 'testing' holdout data subsets.

In these files is the instance's true outcome value, the unique instance ID, and the predicted probability of the instance being case/code 1. 

This code is set up to run on a specific pair of an original dataset and a paired replication dataset one at a time.
 

## Import Packages

In [1]:
import os
import pandas as pd
import pickle
import numpy as np
from statistics import mean
from scipy import interp,stats
import warnings
warnings.filterwarnings('ignore')

# Jupyter Notebook Hack: This code ensures that the results of multiple commands within a given cell are all displayed, rather than just the last. 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Set Run Parameters

In [2]:
experiment_path = "../DemoOutput/demo_experiment"
dataname = 'hcc-data_example_custom' #name of target dataset folder in experiment output folder from pipeline
rep_data_path ="../data/DemoRepData/hcc-data_example_custom_rep.csv"#path to replication dataset file (needed to grab instance labels and true class values)
algorithms = [] #use empty list if user wishes re-evaluate all modeling algorithms that were run in pipeline.

## Load Other Necessary Parameters

In [3]:
# Unpickle metadata from previous phase
file = open(experiment_path+'/'+"metadata.pickle", 'rb')
metadata = pickle.load(file)
file.close()
# Load variables specified earlier in the pipeline from metadata
class_label = metadata['Class Label']
instance_label = metadata['Instance Label']
cv_partitions = int(metadata['CV Partitions'])

# Unpickle algorithm information from previous phase
file = open(experiment_path+'/'+"algInfo.pickle", 'rb')
algInfo = pickle.load(file)
file.close()
algorithms = []
abbrev = {}
colors = {}
for key in algInfo:
    if algInfo[key][0]: # If that algorithm was used
        algorithms.append(key)
        abbrev[key] = (algInfo[key][1])
        colors[key] = (algInfo[key][2])
        
print("Algorithms Ran: " + str(algorithms))

Algorithms Ran: ['Logistic Regression', 'Naive Bayes', 'Decision Tree']


## Extract and Report Case (i.e. class 1) Prediction Probabilities For all instances in replication dataset applied to all CV models.

In [5]:
full_path = experiment_path+'/'+dataname
apply_name = rep_data_path.split('/')[-1].split('.')[0]
new_full_path = full_path+'/replication/'+apply_name
        
#Make folder in experiment folder/datafolder to store all prediction probabilities per algorithm/CV combination
if not os.path.exists(new_full_path+'/model_evaluation/prediction_probas'):
    os.mkdir(new_full_path+'/model_evaluation/prediction_probas')

for algorithm in algorithms: #loop through algorithms
    print(algorithm)

    for cvCount in range(0,cv_partitions): #loop through cv's
        print(cvCount)
        #Load pickled metric file for given algorithm and cv
        result_file = new_full_path+'/model_evaluation/pickled_metrics/'+abbrev[algorithm]+"_CV_"+str(cvCount)+"_metrics.pickle"
        file = open(result_file, 'rb')
        results = pickle.load(file)
        file.close()

        #Load target replication dataset (From which we will get the instancelabel values and class outcome values.)
        rep_data = pd.read_csv(rep_data_path)
        probas_summary = rep_data[[class_label,instance_label]]

        #Separate pickled results
        probas_ = results[9]
        try:
            print(probas_[:,1])
            probas_summary['1_prob']=probas_[:,1]
            file_name = new_full_path+'/model_evaluation/prediction_probas/'+algorithm+'_CV_'+str(cvCount)+'_case_probas.csv'
            probas_summary.to_csv(file_name, index=False)
        except:
            print("prediction_probas not available")

Logistic Regression
0
[1.03633734e-03 8.34763305e-02 9.73280635e-01 7.42604057e-01
 1.64458832e-02 7.47732947e-01 8.79158510e-01 9.81183898e-01
 1.36073845e-01 7.55272443e-03 1.62079473e-01 6.01013680e-07
 2.76302483e-03 7.92617207e-01 3.51759849e-01 6.02499323e-03
 1.18209682e-03 9.96092898e-01 1.98386303e-04 9.99625387e-01
 5.52252123e-03 4.88724783e-04 9.99951846e-01 4.15749383e-01
 2.68619197e-02 9.31414228e-01 1.02797756e-04 1.17221468e-02
 9.72570237e-01 4.05055727e-05 9.98010283e-01 8.08426903e-01
 1.08285829e-01 4.89181599e-02 5.68770320e-05 2.18128649e-01
 1.12501988e-02 3.80280861e-02 2.65776473e-04 1.61146557e-01
 1.63740835e-04 3.09707327e-03 8.14033458e-01 3.54904054e-05
 2.40877018e-04 1.23198152e-01 4.00669712e-01 9.99377239e-01
 3.43778693e-01 3.68303151e-04 9.99996091e-01 9.99923459e-01
 3.65814696e-02 1.46875263e-03 9.99317596e-01 9.87869561e-01
 9.99954126e-01 4.97353317e-02 9.98002036e-01 2.34698956e-03
 1.66309502e-03 2.39700975e-01 1.86862393e-01 2.01095949e-01
 8